#Running simple example through EC2

#### start downloading linking_EC2

Tutorial for running liknking_EC2 see:
https://github.com/JonasWallin/linkingEC2/blob/master/script/running%20MPI4py.ipynb

In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2

  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-FTtSJW-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


In [1]:
from linkingEC2 import LinkingHandler
from ConfigParser import ConfigParser

config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)


linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC' )

##### start the actual cluster

In [2]:

linker.start_cluster('ami-d05e75b8', 't2.micro', ['linking_EC2'],1)
print( linker.get_ssh_login() )



waiting for instances 0  to be running: 
*****

checking if ssh into node001 works:
**
collecting node001 number of processes:done
copying ~/.ssh/mykeyABC.rsa to external ~/.ssh/id_rsa
copying files to node001 done
disable StrictHostKeyChecking  in node001:appending to ~/nodefile in node001 done
appending to /etc/hosts in node001 done
ssh  -i ~/.ssh/mykeyABC.rsa -o 'StrictHostKeyChecking no'  ubuntu@ec2-52-20-99-118.compute-1.amazonaws.com


In [5]:
PACKAGES_APT = [' libatlas3-base',
                'libatlas-base-dev',
                'python-dev', 
                'openmpi-bin', 
                'libopenmpi-dev', 
                'python-numpy',
                'python-sklearn',
               'python-matplotlib',
               'git',
                'python-scipy']
PACKAGES_PIP = ['cython',
                'mpi4py',
                'simplejson']

#Adding later version of R
#linker.send_command_ssh(command = 'sudo gedit /etc/apt/sources.list')
#append line deb http://cran.rstudio.com/bin/linux/ubuntu trusty/
#gpg --keyserver keyserver.ubuntu.com --recv-key E084DAB9
#gpg -a --export E084DAB9 | sudo apt-key add -



linker.apt_install(PACKAGES_APT)

#problem with memory installing scipy:
#http://naokiwatanabe.blogspot.se/2014/12/install-numpy-schipy-matplotlib-and-etc.html
linker.send_command_ssh(command = 'sudo /bin/dd if=/dev/zero of=/var/swap.1 bs=1M count=1024')
linker.send_command_ssh(command = 'sudo /sbin/mkswap /var/swap.1')
linker.send_command_ssh(command = 'sudo /sbin/swapon /var/swap.1')
linker.pip_install('-U scipy')

linker.send_command_ssh(command = 'sudo swapoff /var/swap.1')
linker.send_command_ssh(command = 'sudo sudo rm /var/swap.1')



linker.pip_install(PACKAGES_PIP)
linker.pip_install(['git+https://git@github.com/JonasWallin/BayesFlow'])
import os
os.system('say "your packages is downloaded"')

sshing in: sudo swapoff /var/swap.1 :
sshing in: sudo sudo rm /var/swap.1 :
installing the packages : cython mpi4py simplejson
installing package for node001 done
installing the packages : git+https://git@github.com/JonasWallin/BayesFlow
installing package for node001 done


0

# Terimante the cluster:

In [6]:
linker.terminate_cluster()